In [1]:
from infocoupling_FIR_main_nonipype import decode_FIR_timecourses, save_timeseqs
import os
import pandas as pd

In [2]:
tc = pd.read_csv('/project/3018040.05/FIR_timeseries/test_m29/sub-001_ba-17-18_contr-objscrvsbas_top-500_nothresh.csv')

In [4]:
tc.iloc[0].chosenruns

'[1, 2, 4, 3, 6]'

In [2]:
tc, sub, roi, chosenruns = decode_FIR_timecourses('sub-001', 'ba-17-18_contr-objscrvsbas_top-500_nothresh',
                                                  ('train', 'test'), (5, 29), 'traintest',
                                                  sample_runs=5)

/home/predatt/giaald/.conda/envs/giacomo37/lib/python3.8/site-packages/mvpa2/datasets/base.py:465: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def __init__(self, shape=None, sid=None, fid=None, dtype=np.float):
/home/predatt/giaald/nipype/nipype/utils/misc.py:9: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Iterator
/home/predatt/giaald/nipype/nipype/interfaces/base/traits_extension.py:22: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.

240111-22:03:16,862 nipype.utils WARNING:
	 A newer version (1.8.4) of nipy/nipype is available. You are using 1.6.0-dev+g804adc0


  0%|          | 0/83 [00:00<?, ?it/s]

 * Please note: warnings are printed only once, but underlying problem might occur many times *


100%|██████████| 1169/1169 [10:00<00:00,  1.95it/s]


In [5]:
chosenruns = [1, 2, 4, 3, 6]

In [14]:
tc['chosenruns'] = ''.join((str(c)+',' if i != len(chosenruns)-1 else 
                            str(c) for i, c in enumerate(chosenruns)))

In [15]:
tc

,correct,output,rawdistance,distance,runno,delay,split,view,expected,subject,roi,approach,traintask,testtask,trainmodel,testmodel,chosenruns
0,1,1,-4.378245,1.812118,1,0,1.0,1,True,sub-001,ba-17-18_contr-objscrvsbas_top-500_nothresh,traintest,train,test,5,29,"1,2,4,3,6"
1,0,1,-0.090037,-0.136847,1,0,1.0,1,True,sub-001,ba-17-18_contr-objscrvsbas_top-500_nothresh,traintest,train,test,5,29,"1,2,4,3,6"
2,1,1,-2.300885,1.000558,2,0,1.0,1,True,sub-001,ba-17-18_contr-objscrvsbas_top-500_nothresh,traintest,train,test,5,29,"1,2,4,3,6"
3,1,0,0.072599,-0.073310,2,0,1.0,1,True,sub-001,ba-17-18_contr-objscrvsbas_top-500_nothresh,traintest,train,test,5,29,"1,2,4,3,6"
4,1,1,-1.800194,0.804953,3,0,1.0,1,True,sub-001,ba-17-18_contr-objscrvsbas_top-500_nothresh,traintest,train,test,5,29,"1,2,4,3,6"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,1,0,1.541894,0.786840,3,9,NaN,2,False,sub-001,ba-17-18_contr-objscrvsbas_top-500_nothresh,traintest,train,test,5,29,"1,2,4,3,6"
796,0,0,0.805264,-0.471359,4,9,NaN,2,False,sub-001,ba-17-18_contr-objscrvsbas_top-500_nothresh,traintest,train,test,5,29,"1,2,4,3,6"
797,0,1,-0.668529,-0.159833,4,9,NaN,2,False,sub-001,ba-17-18_contr-objscrvsbas_top-500_nothresh,traintest,train,test,5,29,"1,2,4,3,6"
798,1,1,-0.795194,0.214080,6,9,NaN,2,False,sub-001,ba-17-18_contr-objscrvsbas_top-500_nothresh,traintest,train,test,5,29,"1,2,4,3,6"


In [13]:
outdir = os.path.join('/project/3018040.05/',
                     'FIR_timeseries', 'test_m29')
if not os.path.isdir(outdir):
    os.makedirs(outdir)
tc.to_hdf(os.path.join(outdir, f'{sub}_{roi}.h5'), index=False)